In [39]:
#Dependencies
import pandas as pd
import gensim #the library for Topic modelling
from gensim.models.ldamulticore import LdaMulticore
from gensim import corpora, models


import pyLDAvis.gensim_models
from nltk.corpus import stopwords
import string
from nltk.stem.wordnet import WordNetLemmatizer

import warnings
warnings.simplefilter('ignore')
from itertools import chain


In [16]:
import pandas as pd
import re
pd.options.mode.chained_assignment = None

country = "Australia"
fileName = 'appreciate_campaign_' + country
path = ('campaign_category_results/campaign_category_results/appreciate_campaign/' + fileName + ".csv")

data = pd.read_csv(path)
data_text = data[['Cleaned_Tweet','Tweet_removed_stop_word','sentiment']]
# data_text['ta_tweet'] = data_text['Tweet_removed_stop_word'].apply(lambda x: x[1:-1])
# data_text['ta_tweet'] = data_text['ta_tweet'].str.replace(r"[\"\',]", '',regex=True)
# data_text['ta_tweet'] = data_text['ta_tweet'].str.replace('\'','', regex=True) 


documents = data_text

In [17]:
documents

,Cleaned_Tweet,Tweet_removed_stop_word,sentiment
0,motivational monday start your journey with us...,"['motivational', 'monday', 'start', 'journey',...",5
1,i am sure your patients and administration app...,"['sure', 'patients', 'administration', 'apprec...",5
2,thank you for staying dr ajuwon we appreciate ...,"['thank', 'staying', 'dr', 'ajuwon', 'apprecia...",5
3,i dont know that you should care who the queer...,"['dont', 'know', 'care', 'queer', 'left', 'dis...",2
4,my words have gotten too intense or emotional ...,"['words', 'gotten', 'intense', 'emotional', 'c...",2
...,...,...,...
40767,these stats are absolutely shocking but sittin...,"['stats', 'absolutely', 'shocking', 'sitting',...",3
40768,now onto hendra virus w dr jenny manyweathers ...,"['onto', 'hendra', 'virus', 'w', 'dr', 'jenny'...",1
40769,its more the australian doctors that have done...,"['australian', 'doctors', 'done', 'hard', 'wor...",3
40770,great minds have come together in melbourne to...,"['great', 'minds', 'come', 'together', 'melbou...",5


In [18]:
#clean the data
stop = set(stopwords.words('english'))
exclude = set(string.punctuation)
lemma = WordNetLemmatizer()

def clean(text):
    stop_free = ' '.join([word for word in text.lower().split() if word not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = ' '.join([lemma.lemmatize(word) for word in punc_free.split()])
    return normalized.split()

In [20]:
documents['text_clean']=documents['Cleaned_Tweet'].apply(clean)

In [21]:
documents

,Cleaned_Tweet,Tweet_removed_stop_word,sentiment,text_clean
0,motivational monday start your journey with us...,"['motivational', 'monday', 'start', 'journey',...",5,"[motivational, monday, start, journey, u, like..."
1,i am sure your patients and administration app...,"['sure', 'patients', 'administration', 'apprec...",5,"[sure, patient, administration, appreciate, de..."
2,thank you for staying dr ajuwon we appreciate ...,"['thank', 'staying', 'dr', 'ajuwon', 'apprecia...",5,"[thank, staying, dr, ajuwon, appreciate, wonde..."
3,i dont know that you should care who the queer...,"['dont', 'know', 'care', 'queer', 'left', 'dis...",2,"[dont, know, care, queer, left, dismiss, simpl..."
4,my words have gotten too intense or emotional ...,"['words', 'gotten', 'intense', 'emotional', 'c...",2,"[word, gotten, intense, emotional, carefree, a..."
...,...,...,...,...
40767,these stats are absolutely shocking but sittin...,"['stats', 'absolutely', 'shocking', 'sitting',...",3,"[stats, absolutely, shocking, sitting, infecti..."
40768,now onto hendra virus w dr jenny manyweathers ...,"['onto', 'hendra', 'virus', 'w', 'dr', 'jenny'...",1,"[onto, hendra, virus, w, dr, jenny, manyweathe..."
40769,its more the australian doctors that have done...,"['australian', 'doctors', 'done', 'hard', 'wor...",3,"[australian, doctor, done, hard, work, virus, ..."
40770,great minds have come together in melbourne to...,"['great', 'minds', 'come', 'together', 'melbou...",5,"[great, mind, come, together, melbourne, today..."


In [28]:
df = documents[documents['sentiment'] >= 4].copy(deep=True)

In [29]:
#create dictionary
dictionary = corpora.Dictionary(df['text_clean'])
#Total number of non-zeroes in the BOW matrix (sum of the number of unique words per document over the entire corpus).
print(dictionary.num_nnz)

353811


In [30]:
#create document term matrix
doc_term_matrix = [dictionary.doc2bow(doc) for doc in df['text_clean'] ]
print(len(doc_term_matrix))

19413


In [31]:
lda = gensim.models.ldamodel.LdaModel

In [32]:
num_topics=5
%time ldamodel = lda(doc_term_matrix,num_topics=num_topics,id2word=dictionary,passes=50,minimum_probability=0)

Wall time: 3min 13s


In [33]:
ldamodel.print_topics(num_topics=num_topics)

[(0,
  '0.018*"health" + 0.011*"nurse" + 0.010*"year" + 0.008*"good" + 0.007*"thank" + 0.007*"healthy" + 0.007*"new" + 0.007*"vaccine" + 0.007*"get" + 0.007*"covid19"'),
 (1,
  '0.029*"health" + 0.015*"help" + 0.013*"people" + 0.009*"amp" + 0.008*"need" + 0.008*"mask" + 0.006*"mental" + 0.006*"covid" + 0.006*"care" + 0.006*"get"'),
 (2,
  '0.032*"thank" + 0.018*"safe" + 0.017*"amp" + 0.016*"nurse" + 0.015*"health" + 0.014*"worker" + 0.014*"care" + 0.011*"day" + 0.009*"work" + 0.009*"u"'),
 (3,
  '0.016*"amp" + 0.013*"help" + 0.010*"covid19" + 0.009*"health" + 0.007*"safety" + 0.006*"payment" + 0.006*"one" + 0.006*"quarantine" + 0.006*"paypal" + 0.006*"laybuy"'),
 (4,
  '0.041*"health" + 0.017*"help" + 0.016*"amp" + 0.015*"mental" + 0.011*"covid19" + 0.009*"support" + 0.009*"community" + 0.007*"great" + 0.006*"research" + 0.006*"nursing"')]

In [40]:
lda_display = pyLDAvis.gensim_models.prepare(ldamodel, doc_term_matrix, dictionary, sort_topics=False, mds='mmds')
pyLDAvis.display(lda_display)

In [46]:
# Assigns the topics to the documents in corpus
lda_corpus = ldamodel[doc_term_matrix]

In [49]:
[doc for doc in lda_corpus]


[[(0, 0.95705926),
  (1, 0.01069252),
  (2, 0.0109226685),
  (3, 0.010550163),
  (4, 0.010775393)],
 [(0, 0.19284056),
  (1, 0.020276887),
  (2, 0.7467422),
  (3, 0.020031027),
  (4, 0.02010939)],
 [(0, 0.018473033),
  (1, 0.018425385),
  (2, 0.926193),
  (3, 0.018404355),
  (4, 0.018504178)],
 [(0, 0.14141329),
  (1, 0.22982751),
  (2, 0.5842133),
  (3, 0.022255272),
  (4, 0.022290664)],
 [(0, 0.21365224),
  (1, 0.008879058),
  (2, 0.75987303),
  (3, 0.008785707),
  (4, 0.008809952)],
 [(0, 0.27862003),
  (1, 0.011096349),
  (2, 0.55587524),
  (3, 0.1436486),
  (4, 0.010759811)],
 [(0, 0.033972476),
  (1, 0.033860374),
  (2, 0.86481404),
  (3, 0.033663016),
  (4, 0.033690065)],
 [(0, 0.040674247),
  (1, 0.0400925),
  (2, 0.8391808),
  (3, 0.040003467),
  (4, 0.040048905)],
 [(0, 0.17630327),
  (1, 0.15599464),
  (2, 0.31695127),
  (3, 0.3414467),
  (4, 0.009304139)],
 [(0, 0.025164647),
  (1, 0.025118878),
  (2, 0.89967644),
  (3, 0.025004724),
  (4, 0.025035312)],
 [(0, 0.006916539),

In [47]:
scores = list(chain(*[[score for topic_id,score in topic] \
                      for topic in [doc for doc in lda_corpus]]))

threshold = sum(scores)/len(scores)
print(threshold)

0.19999999996829868


In [48]:
cluster1 = [j for i,j in zip(lda_corpus,df.index) if i[0][1] > threshold]
cluster2 = [j for i,j in zip(lda_corpus,df.index) if i[1][1] > threshold]
cluster3 = [j for i,j in zip(lda_corpus,df.index) if i[2][1] > threshold]
cluster4 = [j for i,j in zip(lda_corpus,df.index) if i[3][1] > threshold]
cluster5 = [j for i,j in zip(lda_corpus,df.index) if i[4][1] > threshold]

In [45]:
df.iloc[cluster2]

IndexError: positional indexers are out-of-bounds